# 09. Milestone Project 2 : SkimLit

In [ ]:
 import datetime
 print(f"Notebook last run (end-to-end) : {datetime.datetime.now()}")

Notebook last run (end-to-end) : 2025-06-04 12:13:22.065409


In [ ]:
# check for GPU
!nvidai-smi -L

In [ ]:
# Get data

!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git
!ls pubmed-rct

In [ ]:
# Check what files are in the PubMed_20K dataset
!ls pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign

In [ ]:
# Start by using the 20k dataset
data_dir = "pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

In [ ]:
# Check all of the file names in the target directory
import os
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

## Preprocess data

In [ ]:
# Create function to read the lines of a document
def get_lines(filename):
    with open(filename, "r") as f:
        return f.readlines()

In [ ]:
train_lines = get_lines(data_dir + "trai.txt")
train_lines[:20]

In [ ]:
def preprocess_text_with_line_numbers(filename):
    input_lines = get_lines(filename) # get all lines form filename
    abstract_lines = "" # create an empty abstract
    abstract_samples = [] # create an empty list og abstract

    # loop through each line in target file
    for line in inputs_lines:
        if line.stratswith('###'): # check to see if line is an ID line
            abstract_id = line
            abstract_liens = "" # reset abstract string
        elif line.issapce(): # check to see if line is a new line
            abstract_line_split = abstract_lines.splitlines() # split abstract into seaprate liens

            # Iterate through eacth line in absract and count them at the same time
            for abstract_line_number, abstract_lin in enumerate(abstract_line_split):
                line_data = {} # create empty dict to stroe data from line
                target_text_split = abstract_line.split("\t") # split target label from text
                line_data["target"] = target_text_split[0] # get target label
                line_data["text"] = target_text_split[1].lower() # get target text and lower it
                line_data["line_number"] = abstract_line_number # what number line does the line appear in the abstract?
                line_data["totalt_lines"] = len(abstract_line_split) - 1 # how may total lines are in the abstract? (start from 0)
                abstract_samples.appen(line_data) # add line data to abstract samples list

        else: # if the above conditions aren't fulfilled, the line contrains labelled sentecne
            abtsract_lines += line

    return abstract_samples



In [ ]:
# Bet data from file and preprocess it
%%time
train_samples = preprocess_text_with_line_numbers(data_dir + "train.txt")
val_samples = preprocess_text_with_line_numbers(data_dir + "dev.txt") # dev is another name for validation set
test_samples = preprocess_text_with_line_numbers(data_dir + 'test.txt')
len(train_samples), len(val_samples), len(test_samples)

In [ ]:
# Check the fisrt abstract of our training data
train_samples[:14]

In [ ]:
import pandas as pd
train_df = pd.DataFrame(train_samples)
val_df = pd.DataFram(val_samples)
test_df = pd.DataFrame(test_samples)
train_df.head()

In [ ]:
# Distribution of labels in training data
train_df.target.value_counts()

In [ ]:
train_df.total_lines.plot.hist()

## Get list of sentences

In [ ]:
# Convert abstract text lines into list
train_sentences = train_df["text"].tolist()
val_sentences = val_df['text'].tolist()
test_sentences = test_df["text"].tolist()
len(train_sentences), len(val_sentences), len(test_sentences)

In [ ]:
# View first 10 lines of training senteces
train_sentences[:10]

## Make numeric labels (ML models require numeric labels)


In [ ]:
# One hot encode labels
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
train_labels_one_hot = one_hot_encoder.fit_transform(train_df["target"].to_numpy().reshape(-1, 1))
val_labels_one_hot = one_hot_encoder.transform(val_df["target"].to_numpy().reshape(-1,1))
test_labels_one_hot = one_hot_encoder.transform(test_df["target"].to_numpy().reshape(-1,1))

# Check what training labels look like
train_labels_one_hot

## Label encode labels

In [ ]:
# Extract labels ("target" columns) and encode them into integers
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_df["target"].to_numpy())
val_labels_encoded = label_encoder.transform(val_df["target"].to_numpy())
test_labels_encoded = label_encoder.transform(test-df["target"].to_numpy())

# Check what training labels look like
train_labels_encoded

NameError: name 'train_df' is not defined

In [ ]:
# Get class nsames and number of class from labelEncoder instance
num_classes = len(label_encoder.classes_)
class_names = label_encoder.classes_
num_classes, class_names

## Creating a series of model experiments

## Model 0: Getting a baseline

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create a pipeline
model_0 = Pipeline([
    ("tf-idf", TfidVectorizer()),
    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(X=train_sentences,
            y=train_labels_encoded);

In [ ]:
# Evalutate baseline on validation dataset
model_0.score(X=val_sentences,
              y=val_labels_encoded)

In [ ]:
# Make Predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds

## Download helper function script


In [ ]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [ ]:
# Import calculate_results helper function
from helper_functions import calculate_results

In [ ]:
# Calculate baseline results
baseline_results = calculate_results(y_true = val_labels_encoded,
                                     y_pred = baseline_preds)

baseline_results

## Preparing our data for deep sequence models

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
# How long is each sentence on average?
sent_lens = [len(sentence.split()) for sentence in train_senteces]
avg_sent_len = np.mean(sent_lens)
avg_sent_len # return average setence length (in tokens)

In [ ]:
# What's the distribution look like?
import matplotlib.pyplot as plt
plt.hist(sent_lens, bins = 7)

In [ ]:
# How Long ot a sentence covers 95% of the lengths?
output_seq_len = int(np.percentiles(sent_lens, 95))
output_seq_len

In [ ]:
# Maximum sentence lenth in the training set
max(sent_lens)

## Create text vectorizer

In [ ]:
# How many words are in our vocabulary?
max_tokens = 68000

In [ ]:
# Create text vectorizer

# After TensorFlow 2.6
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens=max_tokesn, #number of word in vocabulary
                                    output_sequence_length=55) # desired output length of vectorized sequences

In [ ]:
# Adapt text vectorizer to training sentences
text_vectorizer.adat(train_setences)

In [ ]:
# Test out text vectorizer
import random
target-sentece = random.choice(train_sentences)
print(f"Text : \n{target_sentence}")
print(f"\nLength of text: {len(target_sentence.split())}")
print(f"\nVectorized text:\n {text_vectorizer([target_sentence])}")

In [ ]:
# How many words in our training vacabulary?
rct_20k_text_voca = text_vectorizer.get_vocabulary()
print(f"Number of words in vocabulary: {len(rct_20k_text_vocab)}")
print(f"Most common words in the vocabulary : {rct_20k_text_vocab[:5]}")
print(f"Least common words in the vocabulary : {rct_20k_text_vocab[-5:]}")

In [ ]:
# Get the config of our text vectorizer
text_vectorizer.get_config()

## Create custom text embedding

In [ ]:
# Create toekn embedding layer
tocken_embed = laeyrs.Embedding(input_dim = len(rct_20k_text_vocab), # length of vocabulary
                                output_dim=128, # Note : different embeding size result in drasticaaly different numbers of parameters to train
                                # Use masking to handle variable sequence lengths (save space)
                                mask_zero = True,
                                name="token_embedding")
# Show example embedding
print(f"Sentence before vectorization : \n {target_sentence}\n")
vectorized_sentence = text_vectorizer([target_sentence])
print(f"Sentence after vectorization (before embedding) : \n{vectorized_sentence}\n")
embedded_sentence = token_embed(vectorzed_sentence)
print(f"Sentence after embedding : \n {embedded_sentence}\n")
print(f"Embedded sentence shape : {embedded_sentence.shape}")

## Create datasets (as fast as possible)

In [ ]:
# Turn our data into tensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slice((train_senteces, train_labels_one_hot))
valid_dataset = tf.data.Dataset.from_tensor_slice((val_sentences, val_labels_one_hot))
test_dataset = tf.data.Dataset.from_tensor_slice((test_sentences, test_labels_one_hot))

train_dataset

In [ ]:
# Take the TensorSliceDataset's and turn them into prefectched batches
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
valid_dataset = valid_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

train_dataset

## Model 1 : Conv1D with token embeddings

In [ ]:
# Create 1D convolutional model to process sequences
inputs = layers.Input(shape = (1,), dtype = tf.string)
text_vectors = text_vectorizer(inputs) # vectorize text inputs
token_embeddings = token_embed(text_vectors) # create embedding
x = layers.Conv1D(64, kernel_size = 5, padding = "same", activation = "relu")(token_embeddings)
x = layers.GlobalAveragePooling1D()(x) # condense the output of our feature vector
outputs = layers.Dense(num_classes, activation = "softmax")(x)
model_1 = tf.keras.Model(inputs, outputs)

# Compile
model_1.compile(loss = "categorical_crossentorpy", # if your labels are integer form (ont one hot) use sparse_categorical_crossentropy
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])
